## Docker for Python Development

#### How to live 24/7 with Docker 


## About me:

* Miguel Barrientos
* Developer at Piksel
* [github.com/mbarrientos](https://github.com/mbarrientos)


### Content

1. **Docker overview**
  * Docker
  * Docker Compose
2. **Building a Django application**
3. **Thoughts and advices**
4. **Useful images and tools**

# Docker overview

#### Docker
 * Docker is a tool designed to make it easier to create, deploy, and run applications by using containers.

* Lightweight containers allows us to encapsulate a system state.


* We can quickly restore a full environment everytime we need. 

#### Docker Compose

* Tool for defining and running multi-container Docker applications

* Aggregate a set of services (based on Docker images) and manages network visibility 

* Very easy to use.

<center> <img height="60%" src="images/docker-architecture.svg"/></center>

```bash
$ docker run ubuntu:16.04 echo "Hello World"
```

# Building a Django application

A very simple Django project

![alt text](images/project_tree.png)

#### First, we start by creating a **Dockerfile**:
```
FROM python:3.6

RUN apt-get update && \
    apt-get install -y \
        python-psycopg2

WORKDIR /usr/src/app

COPY requirements.txt /usr/src/app
RUN pip install --no-cache-dir -r requirements.txt

COPY . /usr/src/app

EXPOSE 8080
ENTRYPOINT ["python", "manage.py"]
CMD ["runserver", "0.0.0.0:8080"]
```

Notas

To run it, we only need to:

```bash
$ docker build -t my_app .
```

```bash
$ docker run -it -p 8000:8000 my_app 
```

#### Quite simple, right?

Let's add another component to the build.

We create a **docker-compose.yml** file:
```yaml
## Docker Compose
version: '2'
services:
  app:
    build: .
    image: my_app
    ports:
      - "8080:8080"
```

We can start the services with:
```bash
docker-compose up
```

Equivalent to: 
```bash
docker build -t my_app .
docker run -p 8080:8080 my_app
```

Now, we can add a new service and link it to *my_app*

```yaml
## Docker Compose
version: '2'
services:
  app:
    build: .
    image: my_app
    ports:
      - "8080:8080"
    volumes:
      - .:/usr/src/app/
    links:
      - postgres

  postgres:
    image: postgres
    environment:
      - POSTGRES_USER=django
      - POSTGRES_PASSWORD=DockerIsGreat
      - POSTGRES_DB=django
 ```

```yaml
## Docker Compose
version: '2'
services:
  app:
    build: .
    image: my_app
    ports:
      - "8080:8080"
    volumes:
      - .:/usr/src/app/
      - ./logs/django:/var/log/django
    links:
      - postgres
    environment:
      - DB_DEFAULT_HOST=${DB_DEFAULT_HOST}
      - DB_DEFAULT_PORT=${DB_DEFAULT_PORT}
      - DB_DEFAULT_USER=${DB_DEFAULT_USER}
      - DB_DEFAULT_PASSWORD=${DB_DEFAULT_PASSWORD}
      - DB_DEFAULT_NAME=${DB_DEFAULT_NAME}
      - DJANGO_SECRET_KEY=${DJANGO_SECRET_KEY}

  postgres:
    image: postgres
    volumes:
      - ./.data/pgdata:/var/lib/postgresql/data/pgdata
      - ./logs/postgresql:/var/log/postgresql
    environment:
      - POSTGRES_USER=${DB_DEFAULT_USER}
      - POSTGRES_PASSWORD=${DB_DEFAULT_PASSWORD}
      - POSTGRES_DB=${DB_DEFAULT_NAME}
      - PGDATA=/var/lib/postgresql/data/pgdata
 ```

Let's see it in action...

<center> <img src="images/docker-compose-up-hacker.jpg" style="height:600px"/> </center>

### Thoughts and advices

* Mounting your code as a volume is enough to synchronize your container

* Creating an *entry* script is usually a wise choice.

You can even write it in Python (no bash code needed!)

```python
@command(command_type=CommandType.bash)
def manage(*args, **kwargs) -> List[List[str]]:
    cmd = shlex.split(f'python manage.py')
    cmd += args
    return [cmd]

@command(command_type=CommandType.bash)
def unit_tests(*args, **kwargs) -> List[List[str]]:
    tests = shlex.split(f'coverage run --concurrency=multiprocessing manage.py test --parallel')
    tests += args
    return [tests]

@command(command_type=CommandType.bash)
def prospector(*args, **kwargs) -> List[List[str]]:
    cmd = ['prospector']
    cmd += args
    return [cmd]

if __name__ == '__main__':
    sys.exit(Main().run())
```

* No need to work with virtualenvs inside docker.

* Sometimes it's useful to run tests in *library* projects, for encapsulating test environments.

* Remove cache directories for package managers (apt, pip, ...) after building.

* Keep your Dockerfile updated, **DO NOT COMMIT CHANGES**


### Useful images and tools

* Official Python images (https://hub.docker.com/_/python/):
  - python:2.7
  - python:3.6
  - "-slim", "-alpine" versions for really lightweight base images.


* Jupyter notebooks (https://github.com/jupyter/docker-stacks): 

![jupyter_stacks](images/jupyter_stacks.svg)

Out-of-the-box services:
* DBs:
    - postgres
    - mysql
* Streaming & queues:
    - rabbitmq
    - spotify/kafka (kafka + zookeeper)
    - redis
* ML:
    - tensorflow (:latest / :latest-gpu)


Every other "service" you can imagine, try first to *google* for an existing Dockerfile.

#### Rocker 
https://github.com/grammarly/rocker

* Allows us to write *"Rockerfiles"*, enhanced versions of Dockerfile to give some extra features, like:
    - Multi-tagging images.
    - Dockerfile *mutliple-inheritance*.

#### That's all!

<center>
<img src="images/thanks.jpg"/>
</center>